In [1]:
%pip install pycryptodome matplotlib --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 13.3 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 13.4 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 15.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 13.0 MB/s  0:00:01m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 13.6 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8/8 [matplotlib]8 [matplotlib]

[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import secrets

import ed25519
import x25519
import hkdf
import gcm

In [3]:
A_seed_k = secrets.token_bytes(32)

A_pk = ed25519.pk(A_seed_k)
# And makes A_pk public

In [4]:
B_seed_k = secrets.token_bytes(32)

B_pk = ed25519.pk(B_seed_k)
# And makes B_pk public

In [5]:
a = secrets.token_bytes(32)

A = x25519.mult(a, 9)
sig_A = ed25519.sign(A_seed_k, A)
# Sends (A, sig_A) to B     # TODO: How to actually send them together

In [6]:
b = secrets.token_bytes(32)

B = x25519.mult(b, 9)
sig_B = ed25519.sign(B_seed_k, B)
# Send (B, sig_B) to A

In [7]:
# Verify message came from B
def get_common_key(a: int, B: bytes, sig_B: bytes):
    if ed25519.verify(B_pk, B, sig_B) == True:
        common_key = x25519.mult(a, B)
    return common_key

In [8]:
if ed25519.verify(B_pk, B, sig_B) == True:
    A_shared_secret = x25519.mult(a, B)
else:
    raise ValueError("Signiture doesn't verify")

print("Shared secret:", A_shared_secret)

A_session_key = hkdf.hkdf(A_shared_secret, bytes([0x00]), "session key".encode(), L=32)

print("Derived session key:", A_session_key)

Shared secret: b'\xb1\r`}\x90\xf3\xec1\xe4\x16U\xb62\x93\x93\xb05]\x14q\xae<\x07R\xe0O\xdd\xf2\xda\xb7\x06\x1b'
Derived session key: b'\x87\xcd\xd4\xd1/ZV\xf5{\xf5AE\xd1\x1bmp\xea\xf6\xf3\xe6\x8b\x1d\x87Cr\xa0"\xd1\x08\x16!\xb6'


In [9]:
if ed25519.verify(A_pk, A, sig_A) == True:
    B_shared_secret = x25519.mult(b, A)
else:
    raise ValueError("Signiture doesn't verify")

print("Shared secret:", B_shared_secret)

B_session_key = hkdf.hkdf(B_shared_secret, bytes([0x00]), "session key".encode(), L=32)

print("Derived session key:", B_session_key)

Shared secret: b'\xb1\r`}\x90\xf3\xec1\xe4\x16U\xb62\x93\x93\xb05]\x14q\xae<\x07R\xe0O\xdd\xf2\xda\xb7\x06\x1b'
Derived session key: b'\x87\xcd\xd4\xd1/ZV\xf5{\xf5AE\xd1\x1bmp\xea\xf6\xf3\xe6\x8b\x1d\x87Cr\xa0"\xd1\x08\x16!\xb6'


In [10]:
print("Are the derived keys the same?: ", A_session_key == B_session_key)

Are the derived keys the same?:  True


In [11]:
ciphertext, tag = gcm.encrypt("Ana are mere".encode(), A_session_key, secrets.token_bytes(16), None)
print(ciphertext, tag)
# Send (ciphertext, tag) to B

b'\x03\xeb\xafG\x96\xab\xd1\r\xea\x19\xf4\x1a\x83M\xa3ai\xc6L\xff\x01\xfdC\xa7*\xb3\xec$\xc7\xa4\x08\xbb' b'\xb5\x9bN`\xc9a\x86\x0b[%\x0e\xc7\x02_*\xb4'


In [12]:
plaintext = gcm.decrypt(ciphertext, B_session_key, tag, None).decode()
print(plaintext)

Ana are mere
